In [1]:
import os
import sys
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import matplotlib 

matplotlib.rcParams['ps.useafm'] = True
matplotlib.rcParams['pdf.use14corefonts'] = True
matplotlib.rcParams['text.usetex'] = True 

%matplotlib notebook

In [2]:
import pymongo
from pymongo import MongoClient

client = MongoClient("129.26.78.131:27017")

In [3]:
client.database_names()
db = client["bitcoin"]

#BLOCKS = [block for block in db['blocks_1'].find().limit(100)]
#db["test_blocks"].insert(BLOCKS)

In [4]:
db['blocks_1'].find({}).count()

2336

In [5]:
db["blocks_1"].find({'relayed_by':{"$exists":False}}).count()

114

# Aggregation

In [ ]:
for i in range(1, 13):
    db["blocks_{0}".format(i)].aggregate([{"$project":
                                      {"block_index":"$block_index",
                                       "_id":0,
                                       "height":"$block_height",
                                       "relayed_by":"$relayed_by",
                                       "hash": "$hash",
                                       "tx":"$tx"}},
                                 {"$unwind":"$tx"},
                                 {"$project":
                                         {"inputs":"$tx.inputs",
                                          "out":"$tx.out",
                                          "time":"$tx.time",
                                          "hash": "$hash",
                                          "_id":0
                                         }},
                                 {"$unwind":"$inputs"},
                                 {"$project":
                                            {"time":"$time",
                                             "hash": "$hash",
                                             "address_source":"$inputs.prev_out.addr",
                                             "amount":"$inputs.prev_out.value",
                                             "out":"$out"}},
                                 {"$unwind":"$out"},
                                 {"$project":
                                            {"t":"$time",
                                             "tx": "$hash",
                                             "s":"$address_source",
                                             "a_s":"$amount",
                                             "r":"$out.addr",
                                             "tag":"$out.addr_tag_link",
                                             "a_r":"$out.value"}},
                                 {"$out":"blocks{0}_aggregation".format(i)}],allowDiskUse=True)

In [12]:
for a in db["blocks2_aggregation"].find().limit(10):
    print a
    print "\n"

{u'tx_hash': u'00000000000000000271821f46ad96c39cb3968d75e4674ae0e2608187ec0b61', u'address_receiver': u'1BQLNJtMDKmMZ4PyqVFfRuBNvoGhjigBKF', u'_id': ObjectId('594149fdeb89a548daac96c8'), u'amount_received': 1344161049, u'time': 1484764640}


{u'tx_hash': u'00000000000000000271821f46ad96c39cb3968d75e4674ae0e2608187ec0b61', u'amount_received': 1000046, u'address_receiver': u'13iEEwzDVFYBnbzqo1ndJZmsPV86VjV5Ue', u'amount_sent': 3349645, u'time': 1484764518, u'_id': ObjectId('594149fdeb89a548daac96c9'), u'address_source': u'1Ec164DQafR5YLCALhqu5owgEiTZxJZ36G'}


{u'tx_hash': u'00000000000000000271821f46ad96c39cb3968d75e4674ae0e2608187ec0b61', u'amount_received': 16552000, u'address_receiver': u'1489mPccYqHJAV3iPeAkQTP19srQdky8kp', u'amount_sent': 3349645, u'time': 1484764518, u'_id': ObjectId('594149fdeb89a548daac96ca'), u'address_source': u'1Ec164DQafR5YLCALhqu5owgEiTZxJZ36G'}


{u'tx_hash': u'00000000000000000271821f46ad96c39cb3968d75e4674ae0e2608187ec0b61', u'amount_received': 1000046,

# Address Collection

In [ ]:
db["blocks1_aggregation"].aggregate([{"$group":{
                                   "_id":'$address_source',
                                   "transactions":{"$addToSet":{"time":"$time","receiver":"$address_receiver"}}}},
                                   {"$out":"senders"}],allowDiskUse=True)

In [ ]:
db["blocks1_aggregation"].find

In [ ]:
A = [len(a["transactions"]) for a in db["senders"].find()]

In [ ]:
stuff = plt.hist(A[2000:],bins=100)
plt.loglog()

In [ ]:
for a in db["senders"].find().limit(10):
    print a
    print "\n"
    print "\n"